In [1]:
# Import the classes.
from analysis.timing.graph import *
from analysis.timing.entities import *
# Import the timing analysis functions.
import analysis.timing.timing_analysis as analysis


In [2]:
import logging
# Set the logging level.
analysis.set_logger_level(logging.ERROR)

In [3]:
class Device:
    def __init__(self, entities, base_delay=None):
        self.entities = entities
        self.base_delay = base_delay

    def software_delay(self) -> float:
        if self.base_delay:
            return len(self.entities) * self.base_delay
        return 0


def find_containing_node(G: Graph, e: Entity) -> Device:
    for node in G._graph:
        if e in node.get_value().entities:
            return node
    return None


def get_delay(G: Graph, e0: Entity, e1: Entity):
    n0 = find_containing_node(G, e0)
    n1 = find_containing_node(G, e1)
    assert n0, "No node contains {}".format(e0)
    assert n1, "No node contains {}".format(e1)
    spath = G.find_shortest_path(n0, n1)
    assert spath, "Cannot find shortest path"
    d0 = n0.get_value()
    d1 = n1.get_value()
    return spath[0] + d0.software_delay() + d1.software_delay()


def create_relation(G: Graph, e0: Entity, e1: Entity):
    return Relation(e0, e1, get_delay(G, e0, e1))


In [4]:
# Create the nodes.
n0 = Node(0)
n1 = Node(1)
n2 = Node(2)
n3 = Node(3)
n4 = Node(4)

# Create the graph.
G = Graph(
    [
        (n0, n1, 0.07),
        (n1, n2, 0.15),
        (n1, n3, 0.14),
        (n3, n4, 0.06),
        (n2, n3, 0.03),
        (n2, n4, 0.05)
    ],
    directed=False
)


In [5]:
# Load the services.
S0 = Service("S0", boot_del=2, init_del=4, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S1 = Service("S1", boot_del=1, init_del=3, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S2 = Service("S2", boot_del=4, init_del=6, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S3 = Service("S3", boot_del=2, init_del=4, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S4 = Service("S4", boot_del=1, init_del=3, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)

# Load the clients.
C0 = Client("C0", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C1 = Client("C1", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C2 = Client("C2", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C3 = Client("C3", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C4 = Client("C4", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C5 = Client("C5", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)

# Place them inside nodes.
n0.set_value(Device([S0, C0, C1], 0.05))
n1.set_value(Device([S1, S2], 0.025))
n2.set_value(Device([S3, C2, C3], 0.075))
n3.set_value(Device([S4, C4], 0.045))
n4.set_value(Device([C5], 0.076))


In [6]:
# Define the relations between clients and services.
system = [
    create_relation(G, C0, S3),
    create_relation(G, C0, S4),
    create_relation(G, C1, S0),
    create_relation(G, C2, S4),
    create_relation(G, C3, S4),
    create_relation(G, C3, S0),
    create_relation(G, C4, S1),
    create_relation(G, C5, S0)
]

# Compute the worst-case discovery time.
analysis.timing_analysis_system(system)


6.595